<a href="https://colab.research.google.com/github/Angeal13/Text_Generator_ML/blob/master/RNN_PlayGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import keras
from keras.preprocessing import *
import os
import numpy as np

Using TensorFlow backend.


In [0]:
#DataSet
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [0]:
#Load One's own data
#from google.colab import files
#path_to_file=list(files.upload().keys())[0]

In [0]:
#
text=open(path_to_file,'rb').read().decode(encoding='utf-8')

In [0]:
vocab =sorted(set(text))
#Creating a mapping from unique characters to indices
char2idx={u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
#Create a function that turns the text into a list of integers
def text_to_int(text):
  return np.array([char2idx[c] for c in text])
text_as_int=text_to_int(text)

In [0]:
#Fucntion goes Form int to text
def int_to_text(ints):
  try:
    ints =ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [0]:
#Creating a training example
#Create a len sequence to build the epoch
seq_length=100
examples_per_epoch=len(text)//(seq_length+1)

#Creating a training targets
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)

In [0]:
#Create a fucntion that will build teh trainign blocks needed
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text,target_text

dataset=sequences.map(split_input_target) #Use the fucntion on every entry through a map

In [0]:
#Make the Trainig Batches
BATCH_SIZE= 64
VOCAB_SIZE= len(vocab)
EMBEDDING_DIM= 256
RNN_UNITS=1024

BUFFER_SIZE=10000

data=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

In [0]:
#Building the model Using a funcrion
def build_model(vocab_size,embedding_dim,rnn_units, batch_size):
  model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
    tf.keras.layers.LSTM(rnn_units,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,BATCH_SIZE)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) 
  pred = example_batch_predictions[0]

In [0]:
sampled_indices= tf.random.categorical(pred,num_samples=1)

sampled_indices=np.reshape(sampled_indices,(1,-1))[0]
predicted_chars=int_to_text(sampled_indices)

predicted_chars


"FpmptqBI:, izAD.XidQ,zmVySrhq!m,EnVRY3ExHIw&VSHr'Qg&JZ G\nQk,EfdkFdR$&eEud EQdMZJvh'$d-jEh'CgKB'Q;qqw"

In [0]:
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [0]:
model.compile(optimizer='adam',loss=loss)

In [0]:
checkpoint_dir='./training_checkpoints'

checkpoint_prefix=os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
#Trianing the model
history=model.fit(data,epochs=10,callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 28s 161ms/step - loss: 1.1598
Epoch 2/10
172/172 [==============================] - 28s 162ms/step - loss: 1.1223
Epoch 3/10
172/172 [==============================] - 28s 163ms/step - loss: 1.0833
Epoch 4/10
172/172 [==============================] - 28s 163ms/step - loss: 1.0431
Epoch 5/10
172/172 [==============================] - 28s 162ms/step - loss: 1.0011
Epoch 6/10
172/172 [==============================] - 28s 163ms/step - loss: 0.9591
Epoch 7/10
172/172 [==============================] - 28s 162ms/step - loss: 0.9177
Epoch 8/10
172/172 [==============================] - 28s 162ms/step - loss: 0.8754
Epoch 9/10
172/172 [==============================] - 28s 163ms/step - loss: 0.8363
Epoch 10/10
172/172 [==============================] - 28s 162ms/step - loss: 0.7984


In [0]:
#Loading model
model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,batch_size=1)
#Finding latest CheckPoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [0]:
#Funciton that eveluates and generates the text
def generate_text(model,start_string):
  num_generate=800 #Num of charaters to be generated
  #Vectorizing our Start String
  input_eval=[char2idx[s] for s in start_string]
  input_eval=tf.expand_dims(input_eval,0)

  #Empty string to store the results
  text_generated=[]
  #Use a temperature for prediction, the higher the less predictible it becomes
  temperature=1.0
  #Use a model resetin funtion for each Batch size==1
  model.reset_states()
  for i in range(num_generate):
    predictions= model(input_eval)
    #Remove bathc dimentions
    predictions=tf.squeeze(predictions,0)
    #Using a categorical distribution to predict the character returned by the model
    predictions=predictions/temperature
    predicted_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()
    #We pass the predicted character as the next input to model
    #along with the previous hidden states
    input_eval=tf.expand_dims([predicted_id],0)
    text_generated.append(idx2char[predicted_id])
  
  return(start_string+''.join(text_generated))#Returned text Generated by the ML

In [0]:
  inp=input("type a Start String")
  print(generate_text(model,inp))

type a Start StringJulliet
Julliet.

DUKE VINCENTIO:
It is true fast?

POMPEY:
Why, who, sir, that?

Nurse:
Fie't not what I bear king, bid me wear ordarence?
Then let that speak not on thy breath,
Have hat hid moved and rotten for her;
And so are we then, dran matist; what swords he stands,
Though Pesur, and myself are surfer,
And I, that sleewest fld to the word:
Even we as gold and feed, the blows with weed
My labris argue to sea that thou comest.

BIONDELO:
She is to you at home:
Pass not worthy As our were.

PETRUCHIO:
Villain, what blood of eart?

SICINIUS:
Assis the kend.

GLOUCESTER:
Yea, am I lord.

First Servant:
Whop our sister should he be loach'd;
Which is no real that we are;
For he is Norfolk's good: place sorrow in some several turn;
Full ofth show me now, not with a groan:
Master, hath sent for me homicati
